In [22]:
import requests
from API_KEY import API_KEY_ASSEMBLYAI
import time

### Uploading file to server[1]

In [24]:
filename = 'data/GraphAttentionNetworks.wav'
upload_endpoint = 'https://api.assemblyai.com/v2/upload'

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

headers = {'authorization': API_KEY_ASSEMBLYAI}
response = requests.post(upload_endpoint,
                        headers=headers,
                        data=read_file(filename))

print(response.json())

{'upload_url': 'https://cdn.assemblyai.com/upload/4b2c7bc2-cf55-4a30-b854-cefbd46580d5'}


Extract URL

In [16]:
audio_url = response.json()['upload_url']
audio_url

'https://cdn.assemblyai.com/upload/a3066a42-aa57-45c5-996b-1575c62c3daa'

### Transcribe endpoint file[1]

In [17]:
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"
json = { "audio_url": audio_url }

response = requests.post(transcript_endpoint, json=json, headers=headers)
print(response.json())

{'id': '6ejz5eysx0-e0ee-4330-914f-fe4c46eabada', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/a3066a42-aa57-45c5-996b-1575c62c3daa', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'webhook_auth': False, 'webhook_auth_header_name': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': False, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': False, 'content_safety_labels': {}, 'iab_categories_result': {}, 'language_dete

Extract JOB id

In [18]:
job_id = response.json()['id']
job_id

'6ejz5eysx0-e0ee-4330-914f-fe4c46eabada'

### Polling server for response whether JOB already or not

If status is not `completed`, wait for a while then calling request back

In [19]:
polling_endpoint = transcript_endpoint + '/' + job_id
limit_time = 60
while True:
    response = requests.get(polling_endpoint, headers=headers) # getting information to the server
    status = response.json()['status']
    print(f"Current status: {status}")
    if status=='completed' or status=='error' or limit_time<=0 : break
    limit_time-=1
    time.sleep(1)

Current status: processing
Current status: processing
Current status: processing
Current status: completed


### Save transription

In [21]:
if status == 'error': raise Exception('Error File!')
output = 'data/transcript_GraphAttentionNetworks.txt'
with open(output, "w") as ifs:
    ifs.write(response.json()['text'])
print('Transcription saved')

Transcription saved


### References
[[1] How to upload and transcribe file to server](https://www.assemblyai.com/docs/walkthroughs#uploading-local-files-for-transcription)